In [2]:
%matplotlib inline

import pandas as pd
import glob
import os

import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import pearsonr
from scipy.stats import kendalltau
import numpy as np

import scanpy as sc

# settings
plt.rc('font', size = 9)
plt.rc('font', family='sans serif')
plt.rcParams['pdf.fonttype']=42
plt.rcParams['ps.fonttype']=42
plt.rcParams['text.usetex']=False
plt.rcParams['legend.frameon']=False
plt.rcParams['axes.grid']=False
plt.rcParams['legend.markerscale']=0.5
plt.rcParams['savefig.dpi']=600
sns.set_style("ticks")

In [3]:
# load adata
adata = sc.read('/home/ngr4/project/scnd/data/processed/astrocyte_210519.h5ad')

In [7]:
sc.external.tl.harmony_timeseries(adata, tp="timepoint", n_components=500)

findfont: Font family ['Raleway'] not found. Falling back to DejaVu Sans.
findfont: Font family ['Lato'] not found. Falling back to DejaVu Sans.


Running PCA with 500 components
Nearest neighbor computation...


/gpfs/ycga/project/dijk/ngr4/conda_envs/monocle3/lib/python3.8/site-packages/harmony/core.py:70: FutureWarning: This location for 'distances' is deprecated. It has been moved to .obsp[distances], and will not be accesible here in a future version of anndata.
  kNN = temp.uns['neighbors']['distances']


Constucting affinities between 5wk and 12wk...
t+1 neighbors of t...
t neighbors of t+1...
Constucting affinities between 12wk and 18wk...
t+1 neighbors of t...
t neighbors of t+1...
Constucting affinities between 18wk and 24wk...
t+1 neighbors of t...
t neighbors of t+1...
Constucting affinities between 24wk and 30wk...
t+1 neighbors of t...
t neighbors of t+1...


100%|██████████| 500/500 [00:23<00:00, 21.12it/s]

BarnesHut Approximation  took  10.98  seconds
Repulsion forces  took  10.91  seconds
Gravitational forces  took  0.09  seconds
Attraction forces  took  0.86  seconds
AdjustSpeedAndApplyForces step  took  0.44  seconds


In [8]:
# randomly pick from 5wk 
start_cellid = adata.obs.loc[(adata.obs['timepoint']=='5wk'), :].sample(1).index[0]

In [9]:
sc.external.tl.palantir(adata,
    use_adjacency_matrix=True,
    distances_key='harmony_aff_aug')

In [13]:
x = sc.external.tl.palantir_results(adata,
                                early_cell=start_cellid)
adata.obs['pr_pseudotime'] = x.pseudotime

Sampling and flocking waypoints...
Time for determining waypoints: 0.007877727349599203 minutes
Determining pseudotime...
Shortest path distances using 30-nearest neighbor graph...
Time for shortest paths: 0.20574351151784262 minutes
Iteratively refining the pseudotime...
Correlation at iteration 1: 0.9991
Correlation at iteration 2: 0.9999
Entropy and branch probabilities...
Markov chain construction...
Identification of terminal states...
Computing fundamental matrix and absorption probabilities...
Project results to all cells...


In [10]:
sc.tl.tsne(adata, n_pcs=2, use_rep='X_palantir_multiscale', perplexity=150)

In [14]:
# format data like monocle one
dt = pd.DataFrame()
dt['UMAP 1'] = adata.obsm['X_tsne'][:, 0]
dt['UMAP 2'] = adata.obsm['X_tsne'][:, 1]
dt['timepoint'] = adata.obs['timepoint'].to_list()
dt['Pseudotime'] = adata.obs['pr_pseudotime'].to_list()
dt['timepoint_int'] = [int(i.split('wk')[0]) for i in dt['timepoint']]


In [15]:
savefigs = True
cmap_wt = {v:sns.cubehelix_palette(5, start=0.5, rot=-0.75)[i] for i,v in enumerate(['5wk', '12wk', '18wk', '24wk', '30wk'])}


In [16]:
fig, ax = plt.subplots(1, 1, figsize=(5, 4))
sns.scatterplot(dt['UMAP 1'], dt['UMAP 2'], hue=dt['timepoint'], palette=cmap_wt, s=1, rasterized=True, ax=ax)

ax.set_xticks([])
ax.set_yticks([])
ax.set_xlabel('UMAP 1')
ax.legend(bbox_to_anchor=(1.01, 1))

if savefigs:
    fig.savefig('/home/ngr4/project/scnd/results/harmony_umap_bytimepoint.pdf', bbox_inches='tight', dpi=600)
    savefigs = False

/gpfs/ycga/project/dijk/ngr4/conda_envs/monocle3/lib/python3.8/site-packages/seaborn/_decorators.py:36: FutureWarning: Pass the following variables as keyword args: x, y. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  warnings.warn(
findfont: Font family ['Bitstream Vera Sans'] not found. Falling back to DejaVu Sans.
findfont: Font family ['Bitstream Vera Sans'] not found. Falling back to DejaVu Sans.


In [17]:
savefigs = True

In [18]:
fig, ax = plt.subplots(1, 1, figsize=(5, 4))
im = ax.scatter(dt['UMAP 1'], dt['UMAP 2'], c=dt['Pseudotime'], cmap='RdYlBu_r', s=1, rasterized=True)

ax.set_xticks([])
ax.set_yticks([])
ax.set_xlabel('UMAP 1')
ax.set_xlabel('UMAP 2')
cbar = fig.colorbar(im)
cbar.set_label('CellRank Pseudotime', rotation=90)
if savefigs:
    fig.savefig('/home/ngr4/project/scnd/results/harmony_umap_bypseudot.pdf', bbox_inches='tight', dpi=600)
    savefigs = False

In [21]:
savefigs = True
pfp = '/home/ngr4/project/scnd/results/'

In [22]:
fig, ax = plt.subplots(2, 1, figsize=(5, 4))
sns.violinplot(x='timepoint', y='Pseudotime', ax=ax[0], data=dt, dodge=False, hue='timepoint', palette=cmap_wt)
ax[0].set_xlabel('')
ax[0].legend().set_visible(False)
sns.regplot(x='timepoint_int', y='Pseudotime', data=dt, ax=ax[1], scatter=False, color='k')
ax[1].scatter(dt['timepoint_int']+np.random.uniform(-0.1, 0.1, size=(dt.shape[0], )),
           dt['Pseudotime'], 
           c=dt['timepoint'].map(cmap_wt), s=3, alpha=0.8, rasterized=True)
r, p_pearson = pearsonr(dt['timepoint_int'], dt['Pseudotime'])
tau, p_kendall = kendalltau(dt['timepoint_int'], dt['Pseudotime'])
R2 = r**2
ax[0].text(0.2, 0.9, '$tau={:.2e} (P={:.2e})$'.format(tau, p_kendall), transform=ax[0].transAxes, weight='bold',)
ax[1].text(0.6, 0.9, '$R^2={:.2e}$'.format(R2), transform=ax[1].transAxes, weight='bold',)
ax[1].set_ylabel('Monocle3 Pseudotime')
ax[1].set_xlabel('Mouse age (wks)')

if savefigs:
    fig.savefig(os.path.join(pfp, 'violin_scatterplot_harmonypseudotVtimepoint.pdf'), dpi=600, bbox_inches='tight')
    savefigs = False

findfont: Font family ['Bitstream Vera Sans'] not found. Falling back to DejaVu Sans.


In [23]:
savefigs = True

In [24]:
fig, ax = plt.subplots(1, 2, figsize=(8,1))
sns.scatterplot(x=dt['Pseudotime'], y=adata.obsm['1dphate'].squeeze(), hue=adata.obs['timepoint'].to_list(), palette=cmap_wt,
                s=20, alpha=0.5, linewidth=0, ax=ax[0], rasterized=True)
ax[0].set_xlabel('Monocle3 Pseudotime')
ax[0].set_yticks([])
ax[0].legend().set_visible(False)
ax[0].set_title('1D Phate')

sns.scatterplot(x=dt['Pseudotime'], y=np.random.uniform(-1, 1, size=(dt.shape[0],)), hue=adata.obs['timepoint'].to_list(), palette=cmap_wt,
                s=20, alpha=0.5, linewidth=0, ax=ax[1], rasterized=True)
ax[1].set_xlabel('Monocle3 Pseudotime')
ax[1].set_yticks([])
ax[1].legend().set_visible(False)
ax[1].set_title('Jitter')

if savefigs:
    fig.savefig(os.path.join(pfp, 'scatterplots_phateVharmonypseudot.pdf'), dpi=600, bbox_inches='tight')
    savefigs = False